In [4]:
import pandas as pd

# Find the nearby train station for each place
df = pd.read_csv("../data/curated/domain_all_clean.csv")
df.head()

,property_id,street,suburb,state,postcode,latitude,longitude,price,bedrooms,bathrooms,parking,property_type,is_rural,land_size,land_unit,is_retirement,url,price_int
0,13671355,104/300 Swanston Street,MELBOURNE,VIC,3000,-37.810608,144.964584,$550,2,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/104-300-swanston-str...,550.0
1,14535732,708/228 A'BECKETT STREET,MELBOURNE,VIC,3000,-37.810192,144.956600,$500,2,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/708-228-a-beckett-st...,500.0
2,15302782,504/350 La Trobe Street,MELBOURNE,VIC,3000,-37.810680,144.959274,$395 per week,2,1,0,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/504-350-la-trobe-str...,395.0
3,15841654,285 La Trobe Street,MELBOURNE,VIC,3000,-37.810734,144.960815,$65 per week,1,1,1,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/285-la-trobe-street-...,65.0
4,15920687,8092Y/33 Rose Lane,MELBOURNE,VIC,3000,-37.815014,144.953873,$60 per week,1,1,1,Apartment / Unit / Flat,False,0,m²,False,https://www.domain.com.au/8092y-33-rose-lane-m...,60.0


In [5]:
train_df = pd.read_csv("../data/curated/metro.csv")
train_df

,station,the_geom,pids,he_loop,lift
0,Alamein,POINT (145.07955800000002 -37.86884299999997),No,No,No
1,Albion,POINT (144.82470999999998 -37.77765599999998),Dot Matrix,No,No
2,Alphington,POINT (145.031251 -37.77839599999999),Dot Matrix,No,No
3,Altona,POINT (144.830604 -37.86724899999996),LCD,No,No
4,Anstey,POINT (144.96056099999998 -37.761897999999974),No,No,No
...,...,...,...,...,...
214,Williams Landing,POINT (144.74719400000004 -37.87009999999998),Dot Matrix,Yes,Yes
215,Aircraft,POINT (144.760809 -37.866603),No,No,No
216,Flemington Racecourse,POINT (144.907196 -37.787585999999976),No,No,No
217,Showgrounds,POINT (144.914982 -37.783551999999986),No,No,No


In [6]:
lon = train_df.head()["the_geom"]
lat = train_df.head()["the_geom"].str.split(" ").apply(pd.Series)[0]

In [7]:
train_df["lon"] = train_df["the_geom"].str.replace('POINT ', "").str.replace('(', "").str.replace(')', "").str.split(" ").apply(pd.Series)[0]
train_df["lat"] = train_df["the_geom"].str.replace('POINT ', "").str.replace('(', "").str.replace(')', "").str.split(" ").apply(pd.Series)[1]

/var/folders/z6/tzjzcr853jj4ljq6l8whqq0m0000gn/T/ipykernel_50661/4083778682.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_df["lon"] = train_df["the_geom"].str.replace('POINT ', "").str.replace('(', "").str.replace(')', "").str.split(" ").apply(pd.Series)[0]
/var/folders/z6/tzjzcr853jj4ljq6l8whqq0m0000gn/T/ipykernel_50661/4083778682.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_df["lat"] = train_df["the_geom"].str.replace('POINT ', "").str.replace('(', "").str.replace(')', "").str.split(" ").apply(pd.Series)[1]


In [8]:
train_df

,station,the_geom,pids,he_loop,lift,lon,lat
0,Alamein,POINT (145.07955800000002 -37.86884299999997),No,No,No,145.07955800000002,-37.86884299999997
1,Albion,POINT (144.82470999999998 -37.77765599999998),Dot Matrix,No,No,144.82470999999998,-37.77765599999998
2,Alphington,POINT (145.031251 -37.77839599999999),Dot Matrix,No,No,145.031251,-37.77839599999999
3,Altona,POINT (144.830604 -37.86724899999996),LCD,No,No,144.830604,-37.86724899999996
4,Anstey,POINT (144.96056099999998 -37.761897999999974),No,No,No,144.96056099999998,-37.761897999999974
...,...,...,...,...,...,...,...
214,Williams Landing,POINT (144.74719400000004 -37.87009999999998),Dot Matrix,Yes,Yes,144.74719400000004,-37.87009999999998
215,Aircraft,POINT (144.760809 -37.866603),No,No,No,144.760809,-37.866603
216,Flemington Racecourse,POINT (144.907196 -37.787585999999976),No,No,No,144.907196,-37.787585999999976
217,Showgrounds,POINT (144.914982 -37.783551999999986),No,No,No,144.914982,-37.783551999999986


In [9]:
# Call ORS API.
def call_API(property_geo, train_geo):
    locations = [property_geo, train_geo]
    import requests
    import json
    print(locations)
    body = {"locations":locations}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '5b3ce3597851110001cf62487d549e1b81434af0a6e6e18e10f46745',
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

    print(call.status_code, call.reason)
    data = json.loads(call.text)["destinations"]
    print(data)
    return data

In [10]:
property_geo = [145.07955800000002, -37.86884299999997]
train_geo = [144.964584, -37.810608]
data = call_API(property_geo, train_geo)

[[145.07955800000002, -37.86884299999997], [144.964584, -37.810608]]
200 OK
[{'location': [145.079779, -37.868869], 'snapped_distance': 19.6}, {'location': [144.964532, -37.810497], 'snapped_distance': 13.19}]


In [11]:
data

[{'location': [145.079779, -37.868869], 'snapped_distance': 19.6},
 {'location': [144.964532, -37.810497], 'snapped_distance': 13.19}]

In [12]:
geo_df = train_df[["lon", "lat"]]

In [13]:
geo_list = geo_df.values.tolist()

In [14]:
print(property_geo)
count = 0
for i in geo_list:
    print(i)
    call_API(property_geo, i)

[145.07955800000002, -37.86884299999997]
['145.07955800000002', '-37.86884299999997']
[[145.07955800000002, -37.86884299999997], ['145.07955800000002', '-37.86884299999997']]
200 OK
[{'location': [145.079779, -37.868869], 'snapped_distance': 19.6}, {'location': [145.079779, -37.868869], 'snapped_distance': 19.6}]
['144.82470999999998', '-37.77765599999998']
[[145.07955800000002, -37.86884299999997], ['144.82470999999998', '-37.77765599999998']]
200 OK
[{'location': [145.079779, -37.868869], 'snapped_distance': 19.6}, {'location': [144.825003, -37.777514], 'snapped_distance': 30.22}]
['145.031251', '-37.77839599999999']
[[145.07955800000002, -37.86884299999997], ['145.031251', '-37.77839599999999']]
200 OK
[{'location': [145.079779, -37.868869], 'snapped_distance': 19.6}, {'location': [145.031269, -37.778185], 'snapped_distance': 23.55}]
['144.830604', '-37.86724899999996']
[[145.07955800000002, -37.86884299999997], ['144.830604', '-37.86724899999996']]
200 OK
[{'location': [145.079779,

KeyError: 'destinations'